In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
heart=pd.read_csv("../input/heart.csv")
heart.head()

In [ ]:
heart["sex"].groupby(heart["sex"]).count() 

İ think more male in this dataset.

In [ ]:
heart[["target"]].groupby(heart["target"]).count() 

Not well balanced but fairly balanced dataset.

In [ ]:
heart[["target","sex"]].groupby([heart["target"],heart["sex"]]).count() 

I can't understand clearly. I wait I understand. 
Sick people balanced but normal people not  balanced

In [ ]:
import matplotlib.pyplot as plt #import plotting modules
import seaborn as sns
%matplotlib inline

In [ ]:
heart.corr()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(heart.corr(),annot=True)

Slope and oldpeak has negative correlation but not so strong. Just this but I can't delete slope or oldpeak colummn. This dataset is very good.

## We make some plots now. 

In [ ]:
sns.lmplot(data=heart,x="age",y="sex",hue="target")

This plot is very ugly. So age sex is not good metrics for these dataset.

In [ ]:
#age	sex	cp	trestbps	chol	fbs	restecg	thalach	exang	oldpeak	slope	ca	thal
sns.lmplot(data=heart,x="age",y="chol",hue="target")

Plot is good,but Still these data is not separable.

In [ ]:
#age	sex	cp	trestbps	chol	fbs	restecg	thalach	exang	oldpeak	slope	ca	thal
sns.lmplot(data=heart,x="slope",y="chol",hue="target")

Sorry slope is also categrical variable but Interestingly if slope is 2 nearly all time these person is sick.

In [ ]:
sns.lmplot(data=heart,x="thal",y="chol",hue="target")

Thal also a categorical variable. Interestingly thal is 2 nearly all time this person is sick.

I used thal and slope variable for prediction.

In [ ]:
#age	sex	cp	trestbps	chol	fbs	restecg	thalach	exang	oldpeak	slope	ca	thal
sns.lmplot(data=heart,x="trestbps",y="chol",hue="target")

Still not clearly separable. 

## I used Dimension reduction and Plot it with these and see what we see.

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
heart2=heart
x=heart2.drop("target",axis=1)
dimen=pca.fit_transform(x)

In [ ]:
dimen

In [ ]:
newdata=pd.DataFrame(dimen)
newdata.head()

In [ ]:
newdata["target"]=heart["target"]
newdata.head()

In [ ]:
newdata=newdata.rename(index=str, columns={0: "x", 1: "y"})
sns.lmplot(data=newdata,x="x",y="y",hue="target")

It's horrific. Dimension reduction also can't work fine in this purpose. 

## Now we use TSNE

In [ ]:
from sklearn.manifold import TSNE
tsne=TSNE(n_components=2)
dimen=tsne.fit_transform(x)
tsnedata=pd.DataFrame(dimen)
tsnedata["target"]=heart["target"]
tsnedata=tsnedata.rename(index=str, columns={0: "x", 1: "y"})
sns.lmplot(data=tsnedata,x="x",y="y",hue="target")

Maybe more iteration in TSNE work better than this situation. 

In [ ]:
from sklearn.manifold import TSNE
tsne=TSNE(n_components=2,n_iter=2500)
dimen=tsne.fit_transform(x)
tsnedata=pd.DataFrame(dimen)
tsnedata["target"]=heart["target"]
tsnedata=tsnedata.rename(index=str, columns={0: "x", 1: "y"})
sns.lmplot(data=tsnedata,x="x",y="y",hue="target")

Okay with these data i thinm KNN is the best classification method. Because can't easily separable with eyes but more 1 goes more 1 prediction maybe. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import math
k=int(math.sqrt(tsnedata.shape[0])/2)
X=tsnedata.iloc[:,0:2]
Y=tsnedata["target"]
knn=KNeighborsClassifier(n_neighbors=k)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=4331)
knn.fit(X_train,y_train)
ypred=knn.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

Not good enough.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import math
k=int(math.sqrt(tsnedata.shape[0])/2)
X=tsnedata.iloc[:,0:2]
Y=tsnedata["target"]
knn=KNeighborsClassifier(n_neighbors=3)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=4331)
knn.fit(X_train,y_train)
ypred=knn.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

Okay I use random forest KNN can't work very best.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
ypred=rf.predict(X_test)
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

Our old friend SVM maybe good for these type of problem. 

In [ ]:
 from sklearn.svm import SVC

In [ ]:
svm=SVC()
svm.fit(X_train,y_train)
ypred=svm.predict(X_test)
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

Maybe TSNE is not good for classification okay. We use PCA version of this data and classifiy with PCA data

In [ ]:
X=newdata.iloc[:,0:2]
Y=newdata["target"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=4331)
knn=KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train,y_train)
ypred=knn.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

In [ ]:
X=newdata.iloc[:,0:2]
Y=newdata["target"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=4331)
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
ypred=knn.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
ypred=rf.predict(X_test)
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

In [ ]:
svm=SVC()
svm.fit(X_train,y_train)
ypred=svm.predict(X_test)
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

PCA version is not all good.

## Lastly we use raw data without dimension reduction

In [ ]:
heart.head()

In [ ]:
X=heart.iloc[:,0:13]
Y=heart["target"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=4331)
knn=KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train,y_train)
ypred=knn.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

In [ ]:
X=heart.iloc[:,0:13]
Y=heart["target"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=4331)
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
ypred=knn.predict(X_test)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

In [ ]:
svm=SVC()
svm.fit(X_train,y_train)
ypred=svm.predict(X_test)
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
ypred=rf.predict(X_test)
print(accuracy_score(y_test,ypred))
print(confusion_matrix(y_test,ypred))
print(classification_report(y_test,ypred))

### So Random Forest without Dimesion Reduction work just fine. Lastly Medical Data is hard to examine without further Medicine knowledge. More Data is better result. Also Tuning SVM and see good result but I can't have enough time to do this. Thanks for reading :)